# Comprehensive Twitter bot
This notebook will be used to collect and test a plethora of functions that you can add to your Twitter bot. They include, but are not limited to:

* Following all your followers
* Unfollowing everyone you follow
* Liking and retweeting any tweets mentioning you
* Liking and retweeting tweets with selected hashtags in the tweet
* and many more

You are welcome to contribute to this growing list of functions by:

* Forking [this Github repository](https://github.com/adriaan90/twitter-bot)
* Creating pull requests for any changes that you have made



#### Importing required libraries

In [ ]:
import tweepy
import logging
import time
import random
from datetime import datetime, timedelta

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

#### Import the script that creates the Twitter API

In [ ]:
from config import create_api
api = create_api()

#### Like and retweets mentions
This function will search for tweets that mention your Twitter handle and will like and retweet each tweet it finds.

In [ ]:
def fav_retweet(api):
    logger.info('Retrieving tweets...')
    mentions = api.mentions_timeline(tweet_mode = 'extended')
    for mention in reversed(mentions):
        if mention.in_reply_to_status_id is not None or mention.user.id == api.me().id:
            # This tweet is a reply or I'm its author so, ignore it
            return
        
        if not mention.favorited:
            # Mark it as Liked, since we have not done it yet
            try:
                mention.favorite()
                logger.info(f"Liked tweet by {mention.user.name}")
            except Exception as e:
                logger.error("Error on fav", exc_info=True)
                
        if not mention.retweeted:
            # Retweet, since we have not retweeted it yet
            try:
                mention.retweet()
                logger.info(f"Retweeted tweet by {mention.user.name}")
            except Exception as e:
                logger.error("Error on fav and retweet", exc_info=True)


We can test the function by running it and interacting with the bot account `@adriaansbot` to see if the bot picks up tweets mentioning its Twitter handle:

In [ ]:
while True:
    fav_retweet(api)
    logger.info("Waiting...")
    time.sleep(30)

#### Just like and retweet one user's tweets
The function `fav_retweet_user` does the same thing as `fav_retweet`, but rather looks for tweets mentioning another user than youself.

In [ ]:
def fav_retweet_user(api, user_handle):
    search_query = f"{user_handle} -filter:retweets"
    logger.info(f'Retrieving tweets mentioning {user_handle}...')
    tweets = api.search(q=search_query, lang ="en")
    for tweet in tweets:
        if tweet.in_reply_to_status_id is not None or \
            tweet.user.id == api.me().id:
            # This tweet is a reply or I'm its author so, ignore it
            return
        if not tweet.favorited:
            # Mark it as Liked, since we have not done it yet
            try:
                tweet.favorite()
                logger.info(f"Liked a tweet mentioning {user_handle}")
            except Exception as e:
                logger.error("Error on fav", exc_info=True)
        if not tweet.retweeted:
            # Retweet, since we have not retweeted it yet
            try:
                tweet.retweet()
                logger.info(f"Retweeted a tweet mentioning {user_handle}")
            except Exception as e:
                logger.error("Error on fav and retweet", exc_info=True)

Let's test it the same way we tested `fav_retweet`, but now the bot will like and retweet any tweets mentioning my other Twitter account

In [ ]:
while True:
    fav_retweet_user(api, "@Michaelgathara")
    logger.info("Waiting...")
    time.sleep(30)

#### Follow users who follow your Twitter account
This is a nice little function that will follow an account that follows your Twitter account. 

* If the `follow_followers` function is used in a loop, it will be able to follow anyone back as soon as they start following you.
* It can also be used once-off.

In [ ]:
def follow_followers(api):
    logger.info("Retrieving and following followers")
    for follower in tweepy.Cursor(api.followers).items():
        if not follower.following:
            try:
                follower.follow()
                logger.info(f"Following {follower.name}")
            except tweepy.error.TweepError:
                pass

The Twitter account `@adriaansbot` is not following any of its followers, so let us fix that:

In [ ]:
follow_followers(api)

#### Unfollow users on Twitter
The `unfollow` function can be used to:

* Mass un-follow everyone you currently follow. There may not be a lot of scenarios where you want to do that, but you can.
* Unfollow just a certain user. You need to give that user's username, user ID or displayed name as an input to the function (`follower_id`)

This function on default will unfollow everyone you follow, so make sure you use it correctly, or you will have to painfully try and remember who you used to follow...

In [ ]:
def unfollow(api, follower_id = None):
    if not follower_id:
        logger.info("Retrieving current users being followed...")
        for following_id in tweepy.Cursor(api.friends).items():
            try:
                api.destroy_friendship(following_id.id)
                logger.info(f"Unfollowed {following_id.name}")
            except tweepy.error.TweepError:
                pass
    else:
        try:
            api.destroy_friendship(follower_id)
            logger.info(f"Unfollowed {follower_id}...")
        except tweepy.error.TweepError:
            pass            

Let us unfollow just one user that the bot currently follow:

In [ ]:
unfollow(api, "@asvn90")

#### Retweet any tweets with a certain hashtags in the text

When you are interested in certain topics and want to make sure that you engage with other like-minded users on Twitter, you can use the function `retweet_tweets_with_hashtag` to make sure do you do not miss any tweets.

In [ ]:
def retweet_tweets_with_hashtag(api, need_hashtags):
    if type(need_hashtags) is list:
        search_query = f"{need_hashtags} -filter:retweets"
        tweets = api.search(q=search_query, lang ="en", tweet_mode='extended')
        for tweet in tweets:
            hashtags = [i['text'].lower() for i in tweet.__dict__['entities']['hashtags']]
            try:
                need_hashtags = [hashtag.strip('#') for hashtag in need_hashtags]
                need_hashtags = list(need_hashtags)
                if set(hashtags) & set(need_hashtags):
                    if tweet.user.id != api.me().id:
                        api.retweet(tweet.id)
                        logger.info(f"Retweeted tweet from {tweet.user.name}")
                        time.sleep(5)
            except tweepy.TweepError:
                logger.error("Error on retweet", exc_info=True)
    else:
        logger.error("Hashtag search terms needs to be of type list", exc_info=True) 
        return
            

We can test to see if this works by retweeting any tweets that mention the hashtag `#manim`. Make sure to put in a `time.sleep()` in there somewhere as well, otherwise your bot will go crazy with the retweets and Twitter will ban your account.

In [ ]:
while True:
        retweet_tweets_with_hashtag(api, ["#manim"])
        logger.info("Waiting...")
        time.sleep(30)

#### Tweet out a daily tweet
The function will tweet out a string given to it once every day. You can change the frequency by changing the if statement's `timedelta(hours=24)` bit.

In [ ]:
def tweet_daily(api, last_tweeted, text):
    if last_tweeted < datetime.now()-timedelta(hours=24):
        api.update_status(text)
        logger.info(f"Tweeted {text} at {datetime.now().strftime('%m/%d/%Y at %H:%M:%S')}")
        return datetime.now()
    else:
        return last_tweeted

#### Creating a bot for continuous monitoring
The code below is an example of where you will run a `while` loop continuously to do a bunch of jobs. Here the Twitter bot:

* Follow anyone that follows the bot's account
* Tweets a random text from a list provided every day

In [ ]:
def main():
    tweets = ["I am happy", "I am sad", "I am hungry", "I am tired"]
    api = create_api()
    last_tweeted = datetime.now()-timedelta(hours=24)
    while True:
        fav_retweet_user(api,"@asvn90")
        last_tweeted = tweet_daily(api, last_tweeted, random.choice(tweets))
        logger.info("Waiting...")
        time.sleep(60)

In [ ]:
if __name__ == "__main__":
    main()    

#### Retweet any tweets with a certain Ticker Symbol in the text

When you are interested in certain tickers and want to make sure that you engage with other like-minded users on Twitter, you can use the function `retweet_tweets_with_ticker` to make sure do you do not miss any tweets.



In [ ]:
def retweet_tweets_with_ticker(api, need_ticker):
    if type(need_ticker) is list:
        search_query = f"{need_ticker} -filter:retweets"
        tweets = api.search(q=search_query, lang ="en", tweet_mode='extended')
        for tweet in tweets:
            ticker = [i['text'].lower() for i in tweet.__dict__['entities']['symbols']]
            try:
                need_ticker = [hashtag.strip('$') for hashtag in need_ticker]
                need_ticker = list(need_ticker)
                if set(ticker) & set(need_ticker):
                    if tweet.user.id != api.me().id:
                        api.retweet(tweet.id)
                        logger.info(f"Retweeted tweet from {tweet.user.name}")
                        time.sleep(5)
            except tweepy.TweepError:
                logger.error("Error on retweet", exc_info=True)
    else:
        logger.error("Hashtag search terms needs to be of type list", exc_info=True) 
        return           

We can test to see if this works by retweeting any tweets that mention the Ticker `$msft` (Microsoft). Make sure to put in a `time.sleep()` in there somewhere as well, otherwise your bot will go crazy with the retweets and Twitter will ban your account.

In [ ]:
while True:
    retweet_tweets_with_ticker(api, ["$msft"])
    logger.info("Waiting...")
    time.sleep(30)